In [ ]:
%matplotlib notebook
%matplotlib inline

from scipy.stats import chi2, t, norm, sem
import matplotlib.pyplot as plt
import numpy as np

from ipywidgets import interact, interactive, fixed, interact_manual, IntSlider, FloatSlider
import ipywidgets as widgets
from IPython.display import display

# Konfidenzintervalle

In [ ]:
mu = 0
sigma = 1
n = 10
sample_count = 100

def simualte_sampling_from_norm_dist(mu, sd, n, sample_count):
    vals = np.empty([sample_count,n], dtype=float)
    rv = norm(loc = mu, scale = sd)
    for i in range(0,sample_count):
        vals[i,:] = rv.rvs(size = n)
    return vals

def calculate_conf_int(vals, alpha = 0.95):
    dog = vals.size - 1
    sample_mean = np.mean(vals)
    sample_standard_error = sem(vals)
    ci = t.interval(alpha, dog, loc = sample_mean, scale = sample_standard_error)
    return ci 

def is_inside(ci,value):
    if ci[0] <= value <= ci[1]:
        return 1
    else:
        return 0
    
def calc_norm(mu,sigma,k):#x_u,x_o):
    #x = np.linspace(mu - x_u, mu + x_o, 100)
    x = np.linspace(mu - k * sigma, mu + k * sigma, 100)
    y = 1/(sigma * np.sqrt(2 * np.pi)) * np.exp(-(x - mu)**2 / (2 * sigma**2))
    return x,y
    
def plot_confidence_intervals(n,sample_count, conf_level):
    percentile = norm.ppf(1 - (1 - conf_level)/2)
    # Generate data for the Gaussian distribution
    mu_bar, sigma_bar = mu, sigma/np.sqrt(n)
    x,y_norm = calc_norm(mu_bar, sigma_bar, 4)
    x_bar_conf, y_norm_conf = calc_norm(mu_bar, sigma_bar, percentile)
    
    y = simualte_sampling_from_norm_dist(mu, sigma, n, sample_count)
    cis = np.apply_along_axis(calculate_conf_int, axis = 1, arr = y, alpha = conf_level)
    mean = np.apply_along_axis(np.mean, axis = 1, arr = y)
    ci_half_length = mean - cis[:,0]
    mu_inside_ci = np.apply_along_axis(is_inside, axis = 1, arr = cis, value = mu)
    print(f"Anteil der CI's, die mu enthalten: {sum(mu_inside_ci)/sample_count}")
    
    fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(10, 10))
    # Draw the Gaussian distribution
    ax1.plot(x, y_norm, color='blue')
    ax1.fill_between(x_bar_conf, y_norm_conf, color='blue', alpha=0.3)
    
    ax2.errorbar(mean, range(0,sample_count), 
                 xerr=ci_half_length, fmt='o')
    plt.axvline(x=mu, color = 'r')
    plt.show()
    
interact(plot_confidence_intervals, 
         n=IntSlider(min=1, max=50, value=10, continuous_update=False),
         sample_count =IntSlider(min=5, max=1000, value=30, continuous_update=False),
         conf_level=FloatSlider(min=0.001, max=0.999, value=0.95, continuous_update=False))